# Linear Regression

Linear regression finds the linear relationship between variables. In ML it's between features (an input variable) and a label (an output variable).

In [ ]:
import plotly.express as px
import numpy as np
import pandas as pd

# Create dummy data
x = np.linspace(0, 10, 100)
y = np.sin(x)

df = pd.DataFrame({'x': x, 'y': y})

# Create a line plot
fig = px.line(df, x='x', y='y', title='Sine Wave with Plotly')

fig.show()

: 

# Things to think about

- Multi-dimensional linear regression